In [1]:
import cv2 as cv

import sys
sys.path.append('../')  # Adjust path as needed



In [9]:
import cv2 as cv
import mediapipe as mp
from src.fingerspell.ui.common import draw_modal_overlay
from src.fingerspell.core.landmarks import calc_landmark_list
import csv
import tempfile
from src.fingerspell.core.landmarks import pre_process_landmark
from src.fingerspell.ui.user_input import create_label_mapping

import cv2 as cv
import mediapipe as mp
from src.fingerspell.ui.common import draw_modal_overlay
from src.fingerspell.core.landmarks import calc_landmark_list, pre_process_landmark
from src.fingerspell.ui.user_input import create_label_mapping
import csv
import tempfile

from collections import deque
import pandas as pd

import pandas as pd
import csv
import tempfile
from datetime import datetime
from tkinter import filedialog
import tkinter as tk
from pathlib import Path

# User Training Testing

Allow end users to gather their own alphabets and create their own training models


## Functions

In [3]:
def draw_landmarks(image, landmark_point):
    """Draw hand skeleton on image."""
    if len(landmark_point) == 0:
        return image
    
    def draw_line(p1, p2):
        cv.line(image, tuple(landmark_point[p1]), tuple(landmark_point[p2]), (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[p1]), tuple(landmark_point[p2]), (255, 255, 255), 2)
    
    # Thumb
    draw_line(2, 3)
    draw_line(3, 4)
    
    # Index finger
    draw_line(5, 6)
    draw_line(6, 7)
    draw_line(7, 8)
    
    # Middle finger
    draw_line(9, 10)
    draw_line(10, 11)
    draw_line(11, 12)
    
    # Ring finger
    draw_line(13, 14)
    draw_line(14, 15)
    draw_line(15, 16)
    
    # Pinky
    draw_line(17, 18)
    draw_line(18, 19)
    draw_line(19, 20)
    
    # Palm
    draw_line(0, 1)
    draw_line(1, 2)
    draw_line(2, 5)
    draw_line(5, 9)
    draw_line(9, 13)
    draw_line(13, 17)
    draw_line(17, 0)
    
    return image

In [4]:
def discard_samples(temp_filename, alphabet, label_map, collected_per_letter):
    """
    Interactive discard workflow.
    Returns updated collected_per_letter dict.
    """
    import cv2 as cv
    from src.fingerspell.ui.common import draw_modal_input, draw_modal_overlay
    
    # Step 1: Get letter to discard
    cap = cv.VideoCapture(0)
    letter_input = ""
    
    while True:
        ret, image = cap.read()
        if not ret:
            break
        
        image = cv.flip(image, 1)
        image = draw_modal_input(image, "Which letter to discard?", letter_input)
        
        cv.imshow('Discard', image)
        key = cv.waitKey(1)
        
        if key == 27:  # ESC - cancel
            cap.release()
            cv.destroyAllWindows()
            return collected_per_letter
        elif key == 13:  # ENTER
            if letter_input in alphabet:
                break
            else:
                letter_input = ""  # Invalid, clear
        elif key == 8:  # BACKSPACE
            letter_input = letter_input[:-1]
        elif 32 <= key <= 126:
            letter_input += chr(key).upper()
    
    target_letter = letter_input
    current_count = collected_per_letter[target_letter]
    
    # Step 2: Get count to discard
    count_input = ""
    
    while True:
        ret, image = cap.read()
        if not ret:
            break
        
        image = cv.flip(image, 1)
        prompt = f"Discard how many samples of '{target_letter}'? (1-{current_count} or 'all')"
        image = draw_modal_input(image, prompt, count_input)
        
        cv.imshow('Discard', image)
        key = cv.waitKey(1)
        
        if key == 27:  # ESC - cancel
            cap.release()
            cv.destroyAllWindows()
            return collected_per_letter
        elif key == 13:  # ENTER
            if count_input.lower() == 'all':
                discard_count = current_count
                break
            elif count_input.isdigit():
                discard_count = int(count_input)
                if 1 <= discard_count <= current_count:
                    break
            count_input = ""  # Invalid
        elif key == 8:  # BACKSPACE
            count_input = count_input[:-1]
        elif 32 <= key <= 126:
            count_input += chr(key)
    
    # Step 3: Confirm
    confirm_input = ""
    
    while True:
        ret, image = cap.read()
        if not ret:
            break
        
        image = cv.flip(image, 1)
        message = f"Discard {discard_count} samples of '{target_letter}'?\n\nType 'yes' to confirm, ESC to cancel"
        image = draw_modal_overlay(image, message, position='center')
        
        # Show what they're typing
        cv.putText(image, confirm_input, (image.shape[1]//2 - 50, image.shape[0]//2 + 100),
                   cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        
        cv.imshow('Discard', image)
        key = cv.waitKey(1)
        
        if key == 27:  # ESC - cancel
            cap.release()
            cv.destroyAllWindows()
            return collected_per_letter
        elif key == 13:  # ENTER
            if confirm_input.lower() == 'yes':
                break
            confirm_input = ""
        elif key == 8:  # BACKSPACE
            confirm_input = confirm_input[:-1]
        elif 32 <= key <= 126:
            confirm_input += chr(key)
    
    cap.release()
    cv.destroyAllWindows()
    
    # Actually discard from CSV
    print(f"Discarding {discard_count} samples of '{target_letter}'...")
    
    # Read CSV
    df = pd.read_csv(temp_filename, header=None)
    
    # Get label index for this letter
    target_label = label_map[target_letter]
    
    # Find all rows with this label
    mask = df[1] == target_label  # Column 1 is label_index
    matching_indices = df[mask].index.tolist()
    
    # Remove last N samples
    if discard_count >= len(matching_indices):
        # Remove all
        df = df[~mask]
    else:
        # Remove last N
        indices_to_remove = matching_indices[-discard_count:]
        df = df.drop(indices_to_remove)
    
    # Rewrite CSV
    df.to_csv(temp_filename, header=False, index=False)
    
    # Update tracking
    collected_per_letter[target_letter] -= discard_count
    
    print(f"Discarded {discard_count} samples. Remaining for '{target_letter}': {collected_per_letter[target_letter]}")
    
    return collected_per_letter

In [5]:
def save_final_data(temp_filename, alphabet, label_map, output_dir="."):
    """
    Save final training data.
    Strips sample_id column and saves label mapping.
    Handles mixed row lengths (static vs dynamic).
    """
    from datetime import datetime
    from pathlib import Path
    
    print("Reading temp file...")
    
    # Read temp file manually to handle variable column counts
    rows = []
    with open(temp_filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            # Strip sample_id (first column), keep rest
            rows.append(row[1:])
    
    print(f"Read {len(rows)} rows")
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    filename = f"custom_alphabet_keypoints_{timestamp}.csv"
    filepath = Path(output_dir) / filename
    
    # Save keypoints
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(rows)
    
    print(f"Saved keypoints: {filepath.absolute()}")
    
    # Save label mapping
    label_filename = f"custom_alphabet_labels_{timestamp}.csv"
    label_filepath = Path(output_dir) / label_filename
    
    sorted_labels = sorted(label_map.items(), key=lambda x: x[1])
    with open(label_filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        for letter, idx in sorted_labels:
            writer.writerow([letter])
    
    print(f"Saved labels: {label_filepath.absolute()}")
    print("\nSave complete!")
    
    return True

In [10]:
# Create fake temp file for testing
print("Creating fake temp file...")
fake_data = []
test_alphabet = ['A', 'B', 'H']
test_label_map = {'A': 0, 'B': 1, 'H': 2}

# Add some fake static samples (44 columns: sample_id, label, 42 features)
for i in range(5):
    row = [i, 0] + [0.1 * j for j in range(42)]  # A samples
    fake_data.append(row)

for i in range(5, 10):
    row = [i, 1] + [0.2 * j for j in range(42)]  # B samples
    fake_data.append(row)

# Add some fake dynamic samples (86 columns: sample_id, label, 84 features)
for i in range(10, 15):
    row = [i, 2] + [0.3 * j for j in range(84)]  # H samples
    fake_data.append(row)

# Write to temp file
temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv', newline='')
csv_writer = csv.writer(temp_file)
csv_writer.writerows(fake_data)
temp_file.close()

print(f"Created temp file: {temp_file.name}")
print(f"Contains {len(fake_data)} rows")

# Test the save function
print("\nTesting save function...")

# Save to current directory
save_final_data(temp_file.name, test_alphabet, test_label_map)

# Or specify directory
save_final_data(temp_file.name, test_alphabet, test_label_map, output_dir=Path("~/Desktop").expanduser())

Creating fake temp file...
Created temp file: /var/folders/0_/lqcldlm13916lgsd64b4bvx00000gn/T/tmpdme8sod2.csv
Contains 15 rows

Testing save function...
Reading temp file...
Read 15 rows
Saved keypoints: /Users/aaronciuffo/Documents/Education/BUAS/25-26/Week_9_Challenges/fingerspell/notebooks/custom_alphabet_keypoints_20260125_0833.csv
Saved labels: /Users/aaronciuffo/Documents/Education/BUAS/25-26/Week_9_Challenges/fingerspell/notebooks/custom_alphabet_labels_20260125_0833.csv

Save complete!
Reading temp file...
Read 15 rows
Saved keypoints: /Users/aaronciuffo/Desktop/custom_alphabet_keypoints_20260125_0833.csv
Saved labels: /Users/aaronciuffo/Desktop/custom_alphabet_labels_20260125_0833.csv

Save complete!


True

## smoke testing

In [ ]:


# Setup
sample_id = 0
alphabet = ['A', 'B', 'C', 'H', 'J', 'U', 'X', 'Z']
dynamic_letters = {'H', 'J', 'U', 'X', 'Z'}
label_map = create_label_mapping(alphabet)
current_letter = 'A'
is_paused = True
collected = 0
collected_per_letter = {letter: 0 for letter in alphabet}
STATIC_TARGET = 2500  # Change this
DYNAMIC_TARGET = 3500  # Add this

# Rolling buffer for dynamic letters
BUFFER_SIZE = 5
landmark_buffer = deque(maxlen=BUFFER_SIZE)

# Create temp file
temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv', newline='')
csv_writer = csv.writer(temp_file)
print(f"Collecting to: {temp_file.name}")

# Camera setup
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 960)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 540)

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

print(f"Collecting letter: {current_letter}")
print("Press SPACE to start/pause collection")
print("Press ESC to quit")

# Main loop
while True:
    ret, image = cap.read()
    if not ret:
        break
    
    image = cv.flip(image, 1)
    
    if is_paused:
        # Show instructions when paused
        target = DYNAMIC_TARGET if current_letter in dynamic_letters else STATIC_TARGET
        current_count = collected_per_letter[current_letter]
        instructions = f"Ready to collect letter: {current_letter}\n\nPress SPACE to start collecting\nTarget: {current_count}/{target} samples"
        image = draw_modal_overlay(image, instructions, position='center')
    else:
        # Process hand detection
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)
        image_rgb.flags.writeable = True
        
        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            
            # Extract landmarks
            landmark_list = calc_landmark_list(image, hand_landmarks)
            
            # Draw skeleton
            image = draw_landmarks(image, landmark_list)
            
            # Normalize landmarks
            normalized = pre_process_landmark(landmark_list)
            
            # Check if current letter is dynamic
            is_dynamic = current_letter in dynamic_letters
            
            if is_dynamic:
                # Add to buffer
                landmark_buffer.append(normalized)
                
                # Only write if buffer is full
                if len(landmark_buffer) >= BUFFER_SIZE:
                    current = landmark_buffer[-1]
                    old = landmark_buffer[0]
                    
                    # Compute delta features
                    delta = [c - o for c, o in zip(current, old)]
                    
                    # Write: sample_id, label_index, current 42, delta 42 (86 total columns)
                    label_index = label_map[current_letter]
                    csv_writer.writerow([sample_id, label_index] + current + delta)
                    temp_file.flush()
                    sample_id += 1
                    
                    collected += 1
                    collected_per_letter[current_letter] += 1  # Add this
            else:
                # Static letter - write immediately
                # Write: sample_id, label_index, normalized 42 (44 total columns)
                label_index = label_map[current_letter]
                csv_writer.writerow([sample_id, label_index] + normalized)
                temp_file.flush()
                sample_id += 1
                
                collected += 1
                collected_per_letter[current_letter] += 1  # Add this
        
        # Show collection status
        # Show collection status
        target = DYNAMIC_TARGET if current_letter in dynamic_letters else STATIC_TARGET
        current_count = collected_per_letter[current_letter]
        cv.putText(image, f"Collecting {current_letter}: {current_count}/{target}", 
                   (20, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv.imshow('Static Collection', image)
    
    key = cv.waitKey(1)
    
    if key == 27:  # ESC
        print("\nSaving data...")
        temp_file.flush()
        temp_file.close()
        save_final_data(temp_file.name, alphabet, label_map)
        break
    elif key == 32:  # SPACE
        is_paused = not is_paused
        if is_paused:
            landmark_buffer.clear()
        print(f"Collection {'paused' if is_paused else 'started'}")
    elif 97 <= key <= 122:  # lowercase a-z
        letter = chr(key).upper()
        if letter in alphabet:
            current_letter = letter
            is_paused = True
            landmark_buffer.clear()
            print(f"\nSwitched to letter: {current_letter}")
            print(f"Current progress: {collected} samples")
    elif key == ord('D'):  # SHIFT+D
        if is_paused:
            temp_file.flush()  # Make sure file is up to date
            collected_per_letter = discard_samples(temp_file.name, alphabet, label_map, collected_per_letter)
            # Update total collected count
            collected = sum(collected_per_letter.values())
        else:
            print("Pause collection first (press SPACE)")
    elif key == ord('S'):  # SHIFT+S (save and continue)
        print("\nSaving data...")
        temp_file.flush()
        saved = save_final_data(temp_file.name, alphabet, label_map)
        if saved:
            print("Save complete. Collection continues...")

# Cleanup
cap.release()
hands.close()
cv.destroyAllWindows()

# Close and verify file
temp_file.close()
print(f"\nCollection complete!")
print(f"Total samples collected: {collected}")
print(f"Saved to: {temp_file.name}")

with open(temp_file.name, 'r') as f:
    lines = f.readlines()
    print(f"Lines in file: {len(lines)}")
    if lines:
        print(f"First line: {lines[0][:100]}...")
        # Count columns
        first_cols = len(lines[0].split(','))
        print(f"Columns in first line: {first_cols}")

Press SPACE to start/pause collection
Press ESC to quit


I0000 00:00:1769325670.603615       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Collection started
Collection paused
